# Ziegler et al. (2005)

## DEB model calibration

### Importing functions

In [46]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

### This is the DEB model

In [47]:
def DEBmodel (y, t, pars):
    #define initial pools (eu is assumed to be zero)
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3];
    
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    #k=pars[5];
    ce=pars[5];
    MX1=ce/4;
    
    #Scaling function for substrate uptake
    f=S/(Km+S)
    #Fluxes
    uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e + g)
    
    #Define derivatives
    dSdt = -uptake
    dedt = v*(f - e)
    dX1dt = growth*X1 #- k*X1
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1 
           
    return dSdt, dedt, dX1dt, dCO2dt;

### The function below uses the output from DEBmodel to convert biomass pools $e$ and $X_{1}$ to microbial biomass ($B$) and measured biomass proxy-parameter PLFA 

In [73]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, lX1, le
    conversions=pars[5:8]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B) and PLFA
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    PLFA = (conversions[1]/4 + conversions[2]*y[:, 1]/(0.25 + y[:, 1]))*B
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 3].reshape(len(d.Time),1),#CO2
                           PLFA.reshape(len(d.Time),1)), axis=1)

    return yhat

### Objective function is defined

In [74]:
def obj_funDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, lX1, le
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
            
    #e_i = 0.25*((d.ATPinit[0]/d.Cmicinit[0])*pars[6] - pars[7])/(pars[8] - (d.ATPinit[0]/d.Cmicinit[0]))
    X1_i = d.PLFAinit[0]/pars[5]/0.25/pars[6]
    
    y0 = np.array([S_i, 0, X1_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
     
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        #np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.PLFA]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.PLFA)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

### Goodness of fit is calculated

In [75]:
def goodnessDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, lX1, le
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
            
    #e_i = 0.25*((d.ATPinit[0]/d.Cmicinit[0])*pars[6] - pars[7])/(pars[8] - (d.ATPinit[0]/d.Cmicinit[0]))
    X1_i = d.PLFAinit[0]/pars[5]/0.25/pars[6]
    
    y0 = np.array([S_i, 0, X1_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
     

    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        #np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.PLFA]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.PLFA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std    
    Ostd=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.PLFA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [76]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, lX1, le
    conversions=pars[5:8]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B) and PLFA
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    PLFA = (conversions[1]/4 + conversions[2]*y[:, 1]/(0.25 + y[:, 1]))*B
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 2.05, 0.05)),1),#glucose
                           y[:, 1].reshape(len(np.arange(0, 2.05, 0.05)),1),#e
                           y[:, 2].reshape(len(np.arange(0, 2.05, 0.05)),1),#X1
                           B.reshape(len(np.arange(0, 2.05, 0.05)),1),#biomass
                           y[:, 3].reshape(len(np.arange(0, 2.05, 0.05)),1),#CO2
                           #Flush.reshape(len(np.arange(0, 8.05, 0.05)),1),
                           PLFA.reshape(len(np.arange(0, 2.05, 0.05)),1)), axis=1)

    return yhat

In [77]:
#reading the data
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/Ziegler2005.csv', sep=',')
print(d)

                  Study               Soil Substrate  Clay   pH  Ctot  Ntot  \
0  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
1  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
2  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
3  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
4  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
5  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
6  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
7  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   
8  Ziegler et al., 2005  Captina silt loam   Glucose    14  6.6  9.21  0.92   

       Time  PLFAinit     Sinit      PLFA   CO2cumul          S  
0  0.000000  0.052792  42.86916  1.093054   0.000000  42.869160  
1  0.104167  0.052792  42.86916       NaN        NaN  29.552348  
2  0.208333

In [78]:
z_debpars=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             (0, 1), #lX1
                                             (0, 1)]) #le
                                             #(0, 1)]) #iX1

<ipython-input-74-d815a6a6819b>:10: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.PLFAinit[0]/pars[5]/0.25/pars[6]
<ipython-input-47-a3081a537b9a>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-73-d1f0d930d2ee>:14: RuntimeWarning: invalid value encountered in multiply
  PLFA = (conversions[1]/4 + conversions[2]*y[:, 1]/(0.25 + y[:, 1]))*B


In [79]:
print(z_debpars)
print(goodnessDEB(z_debpars.x))

     fun: 7.22005585137262
 message: ['Maximum number of iteration reached']
    nfev: 20312
    nhev: 0
     nit: 1000
    njev: 479
  status: 0
 success: True
       x: array([1.00000000e+00, 4.04058723e+02, 3.95012991e-01, 1.00000000e-15,
       5.35778381e-01, 4.32064783e+00, 0.00000000e+00, 3.87529562e-02])
[ -1.4315105  -25.53086024  67.06172047  20.73877529]


<ipython-input-75-c3ffcf557171>:10: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.PLFAinit[0]/pars[5]/0.25/pars[6]
<ipython-input-47-a3081a537b9a>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
<ipython-input-73-d1f0d930d2ee>:14: RuntimeWarning: invalid value encountered in multiply
  PLFA = (conversions[1]/4 + conversions[2]*y[:, 1]/(0.25 + y[:, 1]))*B


In [80]:
z_debparsDE=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             (0, 1), #lX1
                                             (0, 1)]) #le

In [81]:
print(z_debparsDE)
print(goodnessDEB(z_debparsDE.x))

     fun: 30.201938685834705
     jac: array([-8.88115887e-01, -1.24701569e-04,  7.75379765e-02, -7.46069872e-05,
       -4.86934937e-03, -3.41060515e-05,  4.92189400e-01,  2.20623520e-03])
 message: 'Optimization terminated successfully.'
    nfev: 11634
     nit: 91
 success: True
       x: array([1.00000000e+00, 3.90257983e+03, 7.82063567e+00, 4.84585791e-02,
       2.61217366e-01, 5.03282962e+00, 4.11904583e-04, 3.81793506e-01])
[ 0.6188098  -4.00249714 24.00499428  5.95288666]


In [82]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Ziegler2005Pars.csv', z_debparsDE.x.reshape(1,8), delimiter=",")

## Solution over time for visualization in R

In [83]:
#initial conditions
S_i = d.Sinit[0]
    
X1_i = d.PLFAinit[0]/z_debparsDE.x[5]/0.25/z_debparsDE.x[6]
    
y0 = np.array([S_i, 0, X1_i, 0])

#times
t = np.arange(0, 2.05, 0.05)

#model simulations
Ziegler2005Pred = predDEB(DEBmodel, z_debparsDE.x, t, y0)
   
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Ziegler2005Pred.csv', Ziegler2005Pred, delimiter=",")

# Monod and Pirt models are fitted for comparison to DEB

## Monod model
Model as well as supplementary functions are defined below

In [84]:
def Mmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    k=pars[3];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE)
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [85]:
def calcM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
    ##kPLFA
    conversion=pars[4]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    PLFA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 2].reshape(len(d.Time),1),#CO2
                           PLFA.reshape(len(d.Time),1)), axis=1)

    return yhat

In [86]:
def obj_funM (x):
    #define parameters
    ##v, Km, CUE, k, kPLFA
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
       
    B_i = d.PLFAinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)
    
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        #np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.PLFA]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.PLFA)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [87]:
def goodnessM (x):
    #define parameters
    ##v, Km, CUE, k, kPLFA
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
       
    B_i = d.PLFAinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)
        
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
     

    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        #np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.PLFA]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.PLFA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std    
    Ostd=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.PLFA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [88]:
def predM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    
    #conversion factors
    ##kPLFA
    conversion=pars[4]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    PLFA = conversion*y[:, 1]
    
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 2.05, 0.05)),1),#glucose
                           y[:, 2].reshape(len(np.arange(0, 2.05, 0.05)),1),#CO2
                           PLFA.reshape(len(np.arange(0, 2.05, 0.05)),1)), axis=1)

    return yhat

In [89]:
z_mpars=differential_evolution(obj_funM, [(0.0001, 10000), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (0, 1)]) #kPLFA
                                             #(0, 1)]) #iX1

In [90]:
print(z_mpars)
print(goodnessM(z_mpars.x))

     fun: 30.21694674004387
     jac: array([ 1.17951342e-04, -8.34896544e-05, -1.14752651e-04, -5.04485343e-05,
       -1.31450406e-04])
 message: 'Optimization terminated successfully.'
    nfev: 2178
     nit: 27
 success: True
       x: array([4.76508817e+03, 7.17913301e+03, 8.43612627e-01, 4.47885054e-01,
       4.67536770e-02])
[ 0.47944739 -5.46580243 20.93160485 15.37224822]


In [91]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.PLFAinit[0]/z_mpars.x[4]
    
y0 = np.array([S_i, B_i, 0])
#times
t = np.arange(0, 2.05, 0.05)

#model simulations
Ziegler2005PredM = predM(Mmodel, z_mpars.x, t, y0)
    
#model simulations
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Ziegler2005PredM.csv', Ziegler2005PredM, delimiter=",")

## Pirt model
Model as well as supplementary functions are defined below

In [92]:
def Pmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    m = pars[3];
    k = pars[4];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE) + B*m
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [93]:
def calcP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##kPLFA
    conversion=pars[5]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    PLFA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 2].reshape(len(d.Time),1),#CO2
                           PLFA.reshape(len(d.Time),1)), axis=1)

    return yhat

In [94]:
def obj_funP (x):
    #define parameters
    ##v, Km, CUE, m, k, kPLFA
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.PLFAinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)
        
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        #np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.PLFA]).reshape(len(d.Time),1)),
                     axis=1)

     #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.PLFA)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
       
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [95]:
def goodnessP (x):
    #define parameters
    ##v, Km, CUE, m, k, kPLFA
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.PLFAinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        #np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.PLFA]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                           np.nanmean(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.PLFA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    ##std    
    Ostd=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic[0:18])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.PLFA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [96]:
def predP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##kPLFA
    conversion=pars[5]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    PLFA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 2.05, 0.05)),1),#glucose
                           y[:, 2].reshape(len(np.arange(0, 2.05, 0.05)),1),#CO2
                           PLFA.reshape(len(np.arange(0, 2.05, 0.05)),1)), axis=1)

    return yhat

In [97]:
z_ppars=differential_evolution(obj_funP, [(0.001, 500), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (1e-12, 0.5), #m
                                             (0, 1)]) #kPLFA
                                             #(0, 1)]) #iX1

<ipython-input-94-57aa1f624675>:9: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.PLFAinit[0]/pars[5]
<ipython-input-92-c3d2819ce9aa>:15: RuntimeWarning: invalid value encountered in double_scalars
  respiration = uptake*(1 - CUE) + B*m
<ipython-input-92-c3d2819ce9aa>:20: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death
<ipython-input-93-5361a3edd7bb>:13: RuntimeWarning: invalid value encountered in multiply
  PLFA = conversion*y[:, 1]


In [98]:
print(z_ppars)
print(goodnessP(z_ppars.x))

     fun: 26.062730164429944
     jac: array([ 1.00470663e-03, -6.01473949e-04, -3.25829282e+00, -1.54543045e-04,
        9.98312544e-05, -1.89928073e-03])
 message: 'Optimization terminated successfully.'
    nfev: 3588
     nit: 37
 success: True
       x: array([4.29726440e+02, 7.13351842e+02, 1.00000000e+00, 2.64161277e-01,
       4.20583254e-01, 3.86931241e-02])
[ 0.49737319 -5.27758146 22.55516291 10.28259583]


In [99]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.PLFAinit[0]/z_ppars.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 2.05, 0.05)

#model simulations
Ziegler2005PredP = predP(Pmodel, z_ppars.x, t, y0)
    
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Ziegler2005PredP.csv', Ziegler2005PredP, delimiter=",")